### Trying to solve Task 2 now!

Maybe take a look at [this](https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96) as it is a nice way to show how to handle preprocessing.

In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import multiprocessing
from sklearn.neural_network import MLPClassifier
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
import os
import tempfile

In [2]:
trainFeatUrl = 'https://raw.githubusercontent.com/Toroto006/iml2021/main/task2/train_features.csv?token=AH44KNTLRPLZVY6MBFP2QELAREFCU'
trainLblUrl = 'https://raw.githubusercontent.com/Toroto006/iml2021/main/task2/train_labels.csv?token=AH44KNTFLY4C253ALILRQP3AREFDA'

testFeatUrl = 'https://raw.githubusercontent.com/Toroto006/iml2021/main/task2/test_features.csv?token=AH44KNQXVKJFRL42UMSGLKLAREFCY'

trainFeatPD_raw = pd.read_csv(trainFeatUrl)
trainLblPD = pd.read_csv(trainLblUrl)
testFeatPD = pd.read_csv(testFeatUrl)
print("Start downloading for task2")

Start downloading for task2


First we notice the difference in times, hence change that

In [3]:
np.sort(trainFeatPD_raw["Time"])[-50:]

array([ 31,  31,  31,  31,  32,  32,  32,  32,  32,  32,  33,  33,  33,
        33,  33,  34,  34,  34,  34,  34,  35,  35,  35,  36,  36,  37,
       269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 304,
       305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315])

The align_times does exactly that, as we have "the first 12 hours in ICU". This takes for 300000 entries already ~15 on colab though...

In [4]:
def align_times(fromDF):
  print(f"len of fromDF {len(fromDF)}/12 = {len(fromDF)/12} i.e. len%12 == 0")
  assert len(fromDF)%12 == 0, "We do not have 12 hours for each patient!"

  fromDF = fromDF.sort_values(["pid", "Time"], ascending=[True, True], inplace=False, ignore_index=True)
  for idx, _ in fromDF.iterrows():
    i = idx % 12 + 1
    fromDF.at[idx, 'Time'] = i
  return fromDF

trainFeatPD_raw = align_times(trainFeatPD_raw)
#trainFeatPD_raw.head(2)

len of fromDF 227940/12 = 18995.0 i.e. len%12 == 0


We have an incredible amount of NaN, hence we have to do some more preprocessing.  
Trying to follow [Working with Missing Data in Machine Learning](https://towardsdatascience.com/working-with-missing-data-in-machine-learning-9c0a430df4ce)

In [5]:
percentMiss = trainFeatPD_raw.isnull().sum()/len(trainFeatPD_raw)*100
print(f"Amount of NaNs for some per col in %:\n{percentMiss.tail()}")
print(f"That means we have max {np.sort(trainFeatPD_raw.isnull().sum()/len(trainFeatPD_raw)*100)[-1:]}% missing for some...")

Amount of NaNs for some per col in %:
Heartrate          12.201457
Bilirubin_total    97.663420
TroponinI          98.343424
ABPs               15.920856
pH                 89.012021
dtype: float64
That means we have max [99.68456611]% missing for some...


In [6]:
def lookAge():
  print(f"Our max age is {trainFeatPD_raw['Age'].max()} and min is {trainFeatPD_raw['Age'].min()}.")
  bins = 10
  print(f"Let's go with {bins} bins for histogramm.")
  trainFeatPD_raw.hist(column='Age', bins=bins)

#lookAge()

Looks like we have quite a nice distribution for the age, hence doing some media imputation sounds doable, let's see.  
  
Two options are Simple or KNN. The simple imputer with mean or median does not look good on our data at all, as we have soo much missing data. On the other hand the KNN imputer does not make sense at all as our data is so far apart value wise and hence takes forever to converge, if at all.

In [7]:
def impute(fromDF, constant=True, simple=False, KNN=False):
  if not KNN and not simple and not constant:
    return fromDF
  if KNN:
    imputer = KNNImputer(n_neighbors=2) # Basically never converges
  if simple:
    imputer = SimpleImputer(missing_values=np.nan, strategy='median') # Looks really bad...
  if constant:
    imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
  return pd.DataFrame(imputer.fit_transform(fromDF), columns=fromDF.columns)

#trainFeatPD = impute(trainFeatPD_raw)
#trainFeatPD

Normalizing data necessary/bad/good?

In [8]:
def normalize(fromDF, scalar=None, notIn=["pid", "Time"]):
  listNormalize = [lbl for lbl in fromDF.columns if lbl not in notIn]
  normalized = fromDF.copy() 
  features = normalized[listNormalize]
  # first create scalar
  if scalar is None:
    scalar = StandardScaler()
    scalar.fit(features.values)
  else:
    print("Taking already trained scalar!")
  # do it
  features = scalar.transform(features.values)
  # TODO is it bad to normalize to test data?? https://scikit-learn.org/stable/modules/neural_networks_supervised.html under 1.17.8 they say so
  normalized[listNormalize] = features
  return pd.DataFrame(normalized, index=fromDF.index, columns=fromDF.columns), scalar

#trainFeatPD_normalized, _ = normalize(trainFeatPD_raw)
#trainFeatPD_normalized.head(5)

We came up with a new idea: Do not remove those NaNs in itself, but tell the network which are actually measured and otherwise use the imputed values.

In [9]:
def timeFeaturize(fromDF):
  timeFeatures = [lbl for lbl in fromDF.columns if lbl not in ["pid", "Time", "Age"]]
  count = fromDF[timeFeatures].copy()
  for feat in timeFeatures:
    count.loc[~count[feat].isnull(), feat] = 1  # not nan
    count.loc[count[feat].isnull(), feat] = 0   # nan
  return count

#timeFeaturized = timeFeaturize(trainFeatPD_normalized)

In [10]:
def timeFeaturizeAll(fromDF):
  features = [lbl for lbl in fromDF.columns if lbl not in ["pid", "Time", "Age"]]
  thisTimerizerNaN = {}
  thisTimerizerFeat = {}
  for idx in range(0,12):
    thisTimerizerNaN[idx] = [f"{idx%12+1}_{lbl}_NNaN" for lbl in features]
    thisTimerizerFeat[idx] = [f"{idx%12+1}_{lbl}" for lbl in features]
  
  def fgrp(df):
    #print(df.shape)
    nanns = timeFeaturize(df)
    person = pd.DataFrame(df["Age"].head(1), columns=["Age"])
    # timerize the exists
    for idx, row in nanns.iterrows():
      person[thisTimerizerNaN[idx%12]] = row.values
    # timerize the actual features
    imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0) # Looks really bad...
    imputedFeatures = pd.DataFrame(imputer.fit_transform(df[features]), columns=features)
    for idx, row in imputedFeatures.iterrows():
      person[thisTimerizerFeat[idx%12]] = row.values
    return person

  # from https://stackoverflow.com/questions/26187759/parallelize-apply-after-pandas-groupby
  def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

  return applyParallel(fromDF.groupby("pid"), fgrp)

#timeFeaturizeAll(trainFeatPD_normalized.head(12*3)) # TOO SLOW

In [11]:
def timeFeaturizeAllTwo(fromDF, imputeFeatures=True):
  features = [lbl for lbl in fromDF.columns if lbl not in ["pid", "Time", "Age"]]
  thisTimerizerNaN = {}
  thisTimerizerFeat = {}
  for idx in range(0,12):
    thisTimerizerNaN[idx] = [f"{idx%12+1}_{lbl}_NNaN" for lbl in features]
    thisTimerizerFeat[idx] = [f"{idx%12+1}_{lbl}" for lbl in features]
  thisTimerizer2 = ["pid", "Age"]
  thisTimerizerNaN = [y for x in thisTimerizerNaN.items() for y in x[1]]
  thisTimerizerFeat = [y for x in thisTimerizerFeat.items() for y in x[1]]
  # first two done
  timerized = fromDF[fromDF["Time"] == 1][["pid", "Age"]]
  # exists label
  existTimeFeaturized = timeFeaturize(fromDF)
  rows, _ = existTimeFeaturized.shape
  assert rows % 12 == 0, "ROWS not multiple for patients"
  timerizedNaN = pd.DataFrame(existTimeFeaturized.values.reshape(int(rows/12), 34*12), columns=thisTimerizerNaN)
  #print(timerizedNaN)
  # timerized labels
  if imputeFeatures:
    timerizedLabelFeaturized = impute(fromDF, simple=True, constant=False)[features]
  else:
    timerizedLabelFeaturized = fromDF
  rows, _ = timerizedLabelFeaturized.shape
  assert rows % 12 == 0, "ROWS not multiple for patients"
  timerizedFeat = pd.DataFrame(timerizedLabelFeaturized.values.reshape(int(rows/12), 34*12), columns=thisTimerizerFeat)
  #print(timerizedFeat)
  # combine into one
  otherFeat = pd.concat([timerizedNaN, timerizedFeat], axis=1)
  #print(f"shape timerized: {timerized.shape} and shape otherFeat: {otherFeat.shape}")
  timerized[thisTimerizerNaN+thisTimerizerFeat] = otherFeat.values
  return timerized

#timeFeaturizeAllTwo(trainFeatPD_normalized)

If we use timeFeaturize as our "golden model" as that one is easier to read and understand and compare it to timeFeaturizeTwo, which uses reshape, we see they are the same for the first few patients. This for us means we do the correct thing.

In [12]:
VITALS = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
TESTS = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST','LABEL_Alkalinephos',
         'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
print(f"We have {len(VITALS)} vital labels, {len(TESTS)} test labels and then Sepsis alone.")

We have 4 vital labels, 10 test labels and then Sepsis alone.


Now let's create one preprocessing function, that does all of it to the raw data

In [13]:
def preprocess(fromDF, scalar=None, donormalize=True, doimpute=True):
  timeAligned = align_times(fromDF)
  if donormalize:
    normalizedTimeAligned, scalar = normalize(timeAligned, scalar=scalar)
  else:
    normalizedTimeAligned, scalar = timeAligned, None
  timeFeaturized = timeFeaturizeAllTwo(normalizedTimeAligned, imputeFeatures=doimpute)
  return timeFeaturized, scalar

Now let's try to do a): ORDERING OF MEDICAL TEST  
Binary classification: 0 means that there will be no further tests of this kind ordered whereas 1 means that at least one is ordered in the remaining stay.

The corresponding columns containing the binary ground truth in train_labels are the TESTS array.

For the scoring function use [sklearn.metric.roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score).

In [14]:
def testPreprocess():
  print("Starting preprocessing")
  X, _ = preprocess(trainFeatPD_raw)
  y = trainLblPD.sort_values('pid')
  print(f"Preprocessing done; X.shape: {X.shape} and Y.shape: {y.shape}")

First let's try and do MLPClassifier.

In [15]:
def doMLPClassifier(train_X, train_y):
  print(f"Starting to fit the binary MLPClassifiers!")
  testClf = {}
  i = 0
  for lbl in TESTS:
    clf = MLPClassifier(solver='adam', alpha=1e-5, random_state=42, hidden_layer_sizes=(40, 30, 20, 20))
    clf.fit(train_X.drop("pid", axis='columns'), train_y[lbl])
    testClf[lbl] = clf
    i += 1
    print(f"{lbl} ({i}/{len(TESTS)}) is done!")
  return testClf

def runTestPredictions(testLblClf, test_X):
  print(f"Starting to predict the binary MLPClassifiers!")
  df_submission = {}
  for lbl in TESTS:
    df_submission[lbl] = testLblClf[lbl].predict(test_X.drop("pid", axis='columns'))
  print(f"Done with all of the predictions!")
  return df_submission

#testPreprocess()
#df_submission = runTestPredictions(doMLPClassifier(X, y), X)

In [16]:
def task1Score(actual, submission):
  return np.mean([metrics.roc_auc_score(actual[entry], submission[entry]) for entry in TESTS])

#task1 = task1Score(y, df_submission)
#print(f"Currently task one gives a score of {task1}") # was 0.9987 at some point

Uhh wow, just not adding the pid makes the MLPClassifiers work better, now let's see if it is remembering the data or actually learning:

In [17]:
random_state = 43
# split data and normalize
sorted = trainFeatPD_raw.sort_values(['pid', 'Time'])
rows, cols = sorted.shape
# print(sorted.head(2))
# bring into shape for train_test_split
sameShape = pd.DataFrame(sorted.values.reshape(int(rows/12), 12*37))
# print(sameShape.head(2))
# do the train_test_validation_split
train_data_df, test_data_df = train_test_split(sameShape, test_size=0.2, random_state=random_state)
train_data_df, val_data_df = train_test_split(train_data_df, test_size=0.2, random_state=random_state)
# reverse the shaping to before
rows, _ = test_data_df.shape
test_data_df = pd.DataFrame(test_data_df.values.reshape(rows*12, cols), columns=sorted.columns)
rows, _ = train_data_df.shape
train_data_df = pd.DataFrame(train_data_df.values.reshape(rows*12, cols), columns=sorted.columns)
rows, _ = val_data_df.shape
val_data_df = pd.DataFrame(val_data_df.values.reshape(rows*12, cols), columns=sorted.columns)
# preprocess all of them
train_features, scalar = preprocess(train_data_df)
val_features, _ = preprocess(val_data_df, scalar)
test_features, _ = preprocess(test_data_df, scalar)

len of fromDF 145872/12 = 12156.0 i.e. len%12 == 0
len of fromDF 36480/12 = 3040.0 i.e. len%12 == 0
Taking already trained scalar!
len of fromDF 45588/12 = 3799.0 i.e. len%12 == 0
Taking already trained scalar!


In [18]:
# split labels
y = trainLblPD.sort_values('pid')
train_labels, test_labels = train_test_split(y, test_size=0.2, random_state=random_state)
train_labels, val_labels = train_test_split(train_labels, test_size=0.2, random_state=random_state)
# sort the labels by pid
train_labels = train_labels.sort_values("pid")
val_labels = val_labels.sort_values("pid")
test_labels = test_labels.sort_values("pid")

In [19]:
# to see by eye if it makes sense to now
def test():
  print(train_features.head(1))
  print(train_labels.head(1))
  print(test_features.head(1))
  print(test_labels.head(1))
  print(val_features.head(1))
  print(val_labels.head(1))

#test()

In [20]:
def printShapes():
  print('Training labels shape:', train_labels.shape)
  print('Validation labels shape:', val_labels.shape)
  print('Test labels shape:', test_labels.shape)
  print('Training features shape:', train_features.shape)
  print('Validation features shape:', val_features.shape)
  print('Test features shape:', test_features.shape)

printShapes()

Training labels shape: (12156, 16)
Validation labels shape: (3040, 16)
Test labels shape: (3799, 16)
Training features shape: (12156, 818)
Validation features shape: (3040, 818)
Test features shape: (3799, 818)


Now let's try the MLPClassifiers on our validation set.

In [21]:
def tryMLPC():
  testClf = doMLPClassifier(train_features, train_labels)
  val_sub = runTestPredictions(testClf, val_features)
  test_sub = runTestPredictions(testClf, test_features)
  print(f"task1 score for val: {task1Score(val_labels, val_sub)}")
  print(f"task1 score for test: {task1Score(test_labels, test_sub)}")

#tryMLPC()

As that is barely better than random let's actually try to do a NN.  
For this let's try to follow [imbalanced data](https://www.tensorflow.org/tutorials/structured_data/imbalanced_data) by google.

In [22]:
# dataset imbalance test
for lbl in TESTS:
  neg, pos = np.bincount(y[lbl])
  total = neg + pos
  print(f'Positive: {pos}\t({100 * pos / total}% of total) for {lbl}')
print(f"Total are {total}")

Positive: 5096	(26.828112661226637% of total) for LABEL_BaseExcess
Positive: 1400	(7.3703606212161095% of total) for LABEL_Fibrinogen
Positive: 4554	(23.974730192155832% of total) for LABEL_AST
Positive: 4487	(23.62200579099763% of total) for LABEL_Alkalinephos
Positive: 4570	(24.05896288496973% of total) for LABEL_Bilirubin_total
Positive: 3803	(20.021058173203475% of total) for LABEL_Lactate
Positive: 1895	(9.976309555146091% of total) for LABEL_TroponinI
Positive: 4439	(23.369307712555937% of total) for LABEL_SaO2
Positive: 644	(3.3903658857594103% of total) for LABEL_Bilirubin_direct
Positive: 1254	(6.601737299289287% of total) for LABEL_EtCO2
Total are 18995


Define model and metrics

In [23]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None, last_layer=1):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(400, activation='relu', input_shape=(train_features.drop(columns=["pid"]).shape[-1],), name="layer1"),
      keras.layers.Dropout(0.25),
      keras.layers.Dense(150, activation='relu', name="layer2"),
      keras.layers.Dropout(0.4),
      keras.layers.Dense(last_layer, activation='sigmoid', bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-3),
      loss=keras.losses.binary_crossentropy,
      metrics=metrics)

  return model

In [24]:
EPOCHS = 100
BATCH_SIZE = 1024

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

model = make_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 400)               327200    
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
layer2 (Dense)               (None, 150)               60150     
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 151       
Total params: 387,501
Trainable params: 387,501
Non-trainable params: 0
_________________________________________________________________


Test the keras model.

In [25]:
# to test for single
lbl = TESTS[0]

Set a more correct inital bias (not sure we need this as our data is a lot less imbalanced than the one in the example)

In [26]:
initial_weights = ""
print(lbl)
neg, pos = np.bincount(y[lbl])
total = neg + pos
initial_bias = np.log([pos/neg])
model = make_model(output_bias=initial_bias)
#model.predict(train_features[:10])
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights_'+lbl)
model.save_weights(initial_weights)

LABEL_BaseExcess


In [27]:
model = make_model()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(
    train_features.drop(columns=["pid"]),
    train_labels[lbl],
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features.drop(columns=["pid"]), val_labels[lbl]), 
    verbose=0)

In [28]:
model = make_model()
model.load_weights(initial_weights)
careful_bias_history = model.fit(
    train_features.drop(columns=["pid"]),
    train_labels[lbl],
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features.drop(columns=["pid"]), val_labels[lbl]), 
    verbose=0)

In [29]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_loss(history, label, n):
  # Use a log scale on y-axis to show the wide range of values.
  plt.semilogy(history.epoch, history.history['loss'],
               color=colors[n], label='Train ' + label)
  plt.semilogy(history.epoch, history.history['val_loss'],
               color=colors[n], label='Val ' + label,
               linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()

def plot_comparision():
  plot_loss(zero_bias_history, "Zero Bias", 0)
  plot_loss(careful_bias_history, "Careful Bias", 1)

#plot_comparision()

Clear advantage not taking any initial weights!  
  
Train the model now

In [30]:
baseline_history = {}
models = {}

def trainTest():
  for lbl in TESTS:
    model = make_model()
    model.layers[-1].bias.assign([0.0])
    baseline_history[lbl] = model.fit(
        train_features.drop(columns=["pid"]),
        train_labels[lbl],
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[early_stopping],
        validation_data=(val_features.drop(columns=["pid"]), val_labels[lbl]),
        verbose=0)
    models[lbl] = model
    print(f"Model for {lbl} done!")

#trainTest()

In [31]:
train_predictions_baseline = {}
test_predictions_baseline = {}

def predictTest():
  for lbl in TESTS:
    train_predictions_baseline[lbl] = models[lbl].predict(train_features.drop(columns=["pid"]), batch_size=BATCH_SIZE)
    test_predictions_baseline[lbl] = models[lbl].predict(test_features.drop(columns=["pid"]), batch_size=BATCH_SIZE)
    print(f"Prediction done for {lbl}")

#predictTest()

test all lbls in one

In [32]:
theseTESTS = TESTS + ["LABEL_Sepsis"]
def allTESTS(train_features, train_labels, val_features, val_labels):
  neg, pos = np.bincount(y["LABEL_Sepsis"])
  total = neg + pos
  initial_bias = np.log([pos/neg])
  model = make_model(last_layer=11)
  biases = np.concatenate(([0.0 for _ in range(0,10)], initial_bias))
  model.layers[-1].bias.assign(biases)
  baseline_history = model.fit(
      train_features.drop(columns=["pid"]),
      train_labels[theseTESTS],
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      callbacks=[early_stopping],
      validation_data=(val_features.drop(columns=["pid"]), val_labels[theseTESTS]),
      verbose=0)
  return model

model = allTESTS(train_features, train_labels, val_features, val_labels)

Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping


In [33]:
def doAllTests(model, X_features):
  train_predictions_baseline = model.predict(X_features.drop(columns=["pid"]), batch_size=BATCH_SIZE)
  train_predictions_baseline = pd.DataFrame(train_predictions_baseline, columns=theseTESTS)
  return train_predictions_baseline

train_predictions_baseline = doAllTests(model, train_features)
test_predictions_baseline = doAllTests(model, test_features)

In [34]:
def plot_metrics(history, col, lbl):
  metrics = ['loss', 'prc', 'recall', 'precision']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[col], label=f'Train - {lbl}')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[col], linestyle="--", label=f'Val - {lbl}')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

In [35]:
mpl.rcParams['figure.figsize'] = (20, 18)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  #plt.xlim([-0.5,20])
  #plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [36]:
def roc():
  i = 0
  for lbl in TESTS:
    plot_roc(f"Train Baseline - {lbl}", train_labels[lbl], train_predictions_baseline[lbl], color=colors[i])
    plot_roc(f"Test Baseline - {lbl}", test_labels[lbl], test_predictions_baseline[lbl], color=colors[i], linestyle='--')
    i += 1
    plt.legend(loc='lower right')

#roc()

In [37]:
def otherMeasures():
  i = 0
  for lbl in TESTS:
    plot_metrics(baseline_history[lbl], i, lbl)
    i += 1

# otherMeasures

In [38]:
print(f"For train set I have a task1Score of {task1Score(train_labels, train_predictions_baseline)}") # 0.915 reached with random state 42...
print(f"For test set I have a task1Score of {task1Score(test_labels, test_predictions_baseline)}") # 0.804 reached with random state 42...

For train set I have a task1Score of 0.8971276174163547
For test set I have a task1Score of 0.8003866510612457


Now let's do b), i.e. same binary classification again for sepsis.

Maybe here we could do a test for imbalanced data and actually do the initializiation towards that bias.

In [39]:
lbl = "LABEL_Sepsis"
neg, pos = np.bincount(y[lbl])
total = neg + pos
initial_bias = np.log([pos/neg])
model = make_model(output_bias=initial_bias)
#model.predict(train_features[:10])
initial_weights_sepsis = os.path.join(tempfile.mkdtemp(), 'initial_weights_'+lbl)
model.save_weights(initial_weights_sepsis)

In [40]:
model_sepsis = make_model()
model_sepsis.load_weights(initial_weights_sepsis)
baseline_history_sepsis = model_sepsis.fit(
    train_features.drop(columns=["pid"]),
    train_labels[lbl],
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_features.drop(columns=["pid"]), val_labels[lbl]),
    verbose=0)

Restoring model weights from the end of the best epoch.
Epoch 00026: early stopping


In [41]:
def task2Score(actual, submission):
  return metrics.roc_auc_score(actual['LABEL_Sepsis'], submission)

In [42]:
train_predictions_baseline_sepsis = model_sepsis.predict(train_features.drop(columns=["pid"]), batch_size=BATCH_SIZE)
test_predictions_baseline_sepsis = model_sepsis.predict(test_features.drop(columns=["pid"]), batch_size=BATCH_SIZE)
print(f"Prediction done for LABEL_Sepsis")

Prediction done for LABEL_Sepsis


In [43]:
train_predictions_baseline_sepsis = train_predictions_baseline['LABEL_Sepsis']
test_predictions_baseline_sepsis = test_predictions_baseline["LABEL_Sepsis"]

In [44]:
print(f"For train set I have a task2Score of {task2Score(train_labels, train_predictions_baseline_sepsis)}") # 0.823 reached with random state 42...
print(f"For test set I have a task2Score of {task2Score(test_labels, test_predictions_baseline_sepsis)}") # 0.711 reached with random state 42...

For train set I have a task2Score of 0.8220087365599014
For test set I have a task2Score of 0.7285471905378156


Let's do now the task c):  
Predict a more general evolution of the patient state, i.e. mean value of a vital sign in the remaining stay. This is a regression task.
The corresponding columns are VITALS. To evaluate the performance of a given model on this sub-task we use [R2-Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html).

In [45]:
clf = {}
for lbl in VITALS:
  clf[lbl] = Ridge(alpha=1.0)
  clf[lbl].fit(train_features.drop(columns=["pid"]), train_labels[lbl])
  print(f"{lbl} clf fit!")

LABEL_RRate clf fit!
LABEL_ABPm clf fit!
LABEL_SpO2 clf fit!
LABEL_Heartrate clf fit!


In [46]:
train_predicted = {}
test_predicted = {}
for lbl in VITALS:
  train_predicted[lbl] = clf[lbl].predict(train_features.drop(columns=["pid"]))
  test_predicted[lbl] = clf[lbl].predict(test_features.drop(columns=["pid"]))
  print(f"Prediction of vital {lbl} done")

Prediction of vital LABEL_RRate done
Prediction of vital LABEL_ABPm done
Prediction of vital LABEL_SpO2 done
Prediction of vital LABEL_Heartrate done


In [47]:
def task3Score(actual, submission):
  return np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(actual[entry], submission[entry])) for entry in VITALS])

In [48]:
#print(f"For train set I have a task3Score of {task3Score(train_labels, train_predicted)}")
#print(f"For test set I have a task3Score of {task3Score(test_labels, test_predicted)}")

The following is modified from score_submission.py

In [49]:
def get_score(df_true, df_submission):
    df_submission = df_submission.sort_values('pid')
    df_true = df_true.sort_values('pid')
    task1 = np.mean([metrics.roc_auc_score(df_true[entry], df_submission[entry]) for entry in TESTS])
    task2 = metrics.roc_auc_score(df_true['LABEL_Sepsis'], df_submission['LABEL_Sepsis'])
    task3 = np.mean([0.5 + 0.5 * np.maximum(0, metrics.r2_score(df_true[entry], df_submission[entry])) for entry in VITALS])
    score = np.mean([task1, task2, task3])
    print(f"total score is {score} where the individual scores are {task1}, {task2}, {task3}")
    return score

Let's try to combine all together to see the get_score once:

In [50]:
train_end = dict(train_predictions_baseline, **train_predicted)
test_end = dict(test_predictions_baseline, **test_predicted)
train_end["LABEL_Sepsis"] = train_predictions_baseline_sepsis 
test_end["LABEL_Sepsis"] = test_predictions_baseline_sepsis

In [51]:
train_res = pd.DataFrame(train_features["pid"].values, columns=["pid"])
for lbl in train_end.keys():
  train_res[lbl] = pd.DataFrame(train_end[lbl], columns=[lbl])
test_res = pd.DataFrame(test_features["pid"].values, columns=["pid"])
for lbl in test_end.keys():
  test_res[lbl] = pd.DataFrame(test_end[lbl], columns=[lbl])

In [52]:
print(f"Above was train {get_score(train_labels, train_res)}.")
print(f"Above was test {get_score(test_labels, test_res)}.")

total score is 0.8338999172643687 where the individual scores are 0.8971276174163547, 0.8220087365599014, 0.78256339781685
Above was train 0.8338999172643687.
total score is 0.7596098298022472 where the individual scores are 0.8003866510612457, 0.7285471905378156, 0.7498956478076804
Above was test 0.7596098298022472.


Looks like this might actually work!  
Let's now train each model one last time with all data and then do the final score try.

In [53]:
X, scalar = preprocess(trainFeatPD_raw)
test_X, _ = preprocess(testFeatPD, scalar=scalar)
y = trainLblPD.sort_values('pid')

len of fromDF 227940/12 = 18995.0 i.e. len%12 == 0
len of fromDF 151968/12 = 12664.0 i.e. len%12 == 0
Taking already trained scalar!


Do TESTS final

In [54]:
# split data and normalize
sorted = trainFeatPD_raw.sort_values(['pid', 'Time'])
rows, cols = sorted.shape
# bring into shape for train_test_split
sameShape = pd.DataFrame(sorted.values.reshape(int(rows/12), 12*37))
# do the train_test_validation_split
train_data_df, val_data_df = train_test_split(sameShape, test_size=0.2, random_state=random_state)
# reverse the shaping to before
rows, _ = train_data_df.shape
train_data_df = pd.DataFrame(train_data_df.values.reshape(rows*12, cols), columns=sorted.columns)
rows, _ = val_data_df.shape
val_data_df = pd.DataFrame(val_data_df.values.reshape(rows*12, cols), columns=sorted.columns)
# preprocess all of them
train_features, _ = preprocess(train_data_df, scalar)
val_features, _ = preprocess(val_data_df, scalar)
# split labels
y = trainLblPD.sort_values('pid')
train_labels, val_labels = train_test_split(y, test_size=0.2, random_state=random_state)
# sort the labels by pid
train_labels = train_labels.sort_values("pid")
val_labels = val_labels.sort_values("pid")

len of fromDF 182352/12 = 15196.0 i.e. len%12 == 0
Taking already trained scalar!
len of fromDF 45588/12 = 3799.0 i.e. len%12 == 0
Taking already trained scalar!


In [55]:
baseline_history = {}
models = {}
  
def doOld():
  for lbl in TESTS:
    model = make_model()
    model.layers[-1].bias.assign([0.0])
    baseline_history[lbl] = model.fit(
        train_features.drop(columns=["pid"]),
        train_labels[lbl],
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[early_stopping],
        validation_data=(val_features.drop(columns=["pid"]), val_labels[lbl]),
        verbose=0)
    models[lbl] = model
    print(f"Model for {lbl} done!")
    
# doOld()

In [56]:
def doOldPredict():
  tests_predicted = {}
  for lbl in TESTS:
    tests_predicted[lbl] = models[lbl].predict(test_X.drop(columns=["pid"]), batch_size=BATCH_SIZE)
    print(f"Prediction done for {lbl}")

# doOldPredict()

Do Sepsis final

In [57]:
# This makes an initial bias
lbl = "LABEL_Sepsis"
neg, pos = np.bincount(y[lbl])
total = neg + pos
initial_bias = np.log([pos/neg])
model = make_model(output_bias=initial_bias)
#model.predict(train_features[:10])
initial_weights_sepsis = os.path.join(tempfile.mkdtemp(), 'initial_weights_'+lbl)
model.save_weights(initial_weights_sepsis)

In [58]:
def doOldSepsis():
  model_sepsis = make_model()
  model_sepsis.load_weights(initial_weights_sepsis)
  baseline_history_sepsis = model_sepsis.fit(
      train_features.drop(columns=["pid"]),
      train_labels[lbl],
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      callbacks=[early_stopping],
      validation_data=(val_features.drop(columns=["pid"]), val_labels[lbl]),
      verbose=0)
  return model_sepsis

#model_sepsis = doOldSepsis()

In [59]:
def predictOldSepsis():
  sepsis_predicted = model_sepsis.predict(test_X.drop(columns=["pid"]), batch_size=BATCH_SIZE)
  print(f"Prediction done for LABEL_Sepsis")
  return sepsis_predicted

#sepsis_predicted = predictOldSepsis()

Try one go with one model for TESTS+sepsis

In [60]:
model = allTESTS(train_features, train_labels, val_features, val_labels)

Restoring model weights from the end of the best epoch.
Epoch 00023: early stopping


In [61]:
tests_predicted = doAllTests(model, test_X)
tests_predicted

,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis
0,0.782933,0.672191,0.978571,0.987618,0.985059,0.661743,0.013152,0.430337,0.401781,0.021800,0.120071
1,0.014278,0.011855,0.190448,0.193228,0.205053,0.036682,0.109982,0.034584,0.010962,0.005575,0.024984
2,0.060708,0.039840,0.213591,0.216848,0.208401,0.073829,0.101047,0.102489,0.024874,0.013908,0.025436
3,0.473054,0.848914,0.981806,0.985494,0.985189,0.752709,0.032107,0.605220,0.482894,0.065916,0.087024
4,0.180231,0.053243,0.150614,0.152541,0.144320,0.108486,0.022076,0.090892,0.014362,0.004558,0.034053
...,...,...,...,...,...,...,...,...,...,...,...
12659,0.037563,0.029785,0.221136,0.197628,0.210160,0.032462,0.012869,0.019388,0.009874,0.001925,0.018005
12660,0.801476,0.143467,0.352687,0.320399,0.293583,0.615172,0.057113,0.462550,0.050652,0.053806,0.088313
12661,0.844711,0.011014,0.016583,0.027093,0.019256,0.242190,0.002726,0.559636,0.001666,0.000469,0.029351
12662,0.007349,0.020716,0.367023,0.354466,0.395569,0.052729,0.215693,0.034549,0.024417,0.047700,0.043697


Do VITALS final

In [62]:
clf = {}
for lbl in VITALS:
  clf[lbl] = Ridge(alpha=1.0)
  clf[lbl].fit(X.drop(columns=["pid"]), y[lbl])
  print(f"{lbl} clf fit!")

LABEL_RRate clf fit!
LABEL_ABPm clf fit!
LABEL_SpO2 clf fit!
LABEL_Heartrate clf fit!


In [63]:
vitals_predicted = {}
for lbl in VITALS:
  vitals_predicted[lbl] = clf[lbl].predict(test_X.drop(columns=["pid"]))
  print(f"Prediction of vital {lbl} done")

Prediction of vital LABEL_RRate done
Prediction of vital LABEL_ABPm done
Prediction of vital LABEL_SpO2 done
Prediction of vital LABEL_Heartrate done


Combine them all

In [64]:
final_end = dict(tests_predicted, **vitals_predicted)
#final_end["LABEL_Sepsis"] = sepsis_predicted

In [65]:
res = pd.DataFrame(test_X["pid"].values, columns=["pid"])
for lbl in final_end.keys():
  res[lbl] = pd.DataFrame(final_end[lbl], columns=[lbl])
#res

This is to output the final zip to be downloaded

In [66]:
filename = "finalOut.zip"
res.to_csv(filename, index=False, float_format='%.3f', compression='zip')

Run the score_submission to test if it works.

In [67]:
# filename = 'sample.csv'
df_submission = pd.read_csv(filename)

# generate a baseline based on sample.zip
df_true = pd.read_csv(filename)
for label in TESTS + ['LABEL_Sepsis']:
    # round classification labels
    df_true[label] = np.around(df_true[label].values)

get_score(df_true, df_submission)

total score is 1.0 where the individual scores are 1.0, 1.0, 1.0


1.0